Initial ideas:
- feature joining last transaction data with current one (time of purchase, location)
- additional info based on the location of the transaction - if previous purchase was made in Portugal 2h ago, is it possible that the current one is made in Poland? probably not

In [82]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
df_transactions = pd.read_json('../data/transactions.json', lines=True)
df_users = pd.read_csv('../data/users.csv')
df_merchants = pd.read_csv('../data/merchants.csv')

In [84]:
df_transactions.head(1)

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,is_international,session_length_seconds,is_first_time_merchant,is_fraud
0,TX000000,2022-06-17 23:28:00,U14804,M0314,130.03,in-store,EUR,Android,"{'lat': 40.057938, 'long': 14.959737}",debit_card,1,145,0,0


In [85]:
df_merchants.head(1)

,merchant_id,category,country,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history
0,M0001,travel,Austria,1.0,3,97.23,84,0


In [86]:
df_users.head(1)

,user_id,age,sex,education,primary_source_of_income,sum_of_monthly_installments,sum_of_monthly_expenses,country,signup_date,risk_score
0,U00001,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079


# Feature engineering

### 🕒 Temporal Features

| Feature Name                                | Format             | Use-Case                                                                 |
|--------------------------------------------|--------------------|--------------------------------------------------------------------------|
| hour of day                                 | Integer (0–23)     | Captures time-of-day behavior; useful for detecting night-time fraud     |
| day of week                                 | Integer (0–6)      | Weekly behavioral patterns; certain days may have higher fraud rates     |
| is weekend                                  | Boolean            | Distinguishes weekend activity; weekends often have different behavior   |
| month of transaction                        | Integer (1–12)     | Seasonal trends; useful for detecting periodic fraud campaigns           |
| time since last transaction - user-level    | Float (seconds)    | Captures user activity frequency; useful for behavioral profiling        |
| time since last transaction - merchant-level| Float (seconds)    | Captures merchant popularity; good for modeling merchant-specific fraud  |
| time since last user-merchant interaction   | Float (seconds)    | Detects anomalies in specific user-merchant interaction patterns         |

---

### 📊 Spatial Features
| Feature Name                               | Format             | Use-Case                                                                 |
|--------------------------------------------|--------------------|--------------------------------------------------------------------------|
| latitude and longitude extracted from dict | Float     | further feature engineering         |
| distance from last user transaction | Float (km)     |          |
| avg velocity between user transactions | Float (km/h)     | its not usually possible to travel 5000 km in 1 hour |

In [87]:
# --------- Temporal data feature engineering

df_transactions['hour'] = df_transactions['timestamp'].dt.hour
df_transactions['day_of_week'] = df_transactions['timestamp'].dt.dayofweek  # 0=Monday
df_transactions['is_weekend'] = df_transactions['day_of_week'] >= 5
df_transactions['month'] = df_transactions['timestamp'].dt.month

# Time Since Last Transaction: User-level
df_transactions = df_transactions.sort_values(['user_id', 'timestamp'])
df_transactions['time_since_last_user_txn'] = (
    df_transactions.groupby('user_id')['timestamp']
    .diff()
    .dt.total_seconds()
)

# Time Since Last Transaction: Merchant-level
df_transactions = df_transactions.sort_values(['merchant_id', 'timestamp'])
df_transactions['time_since_last_merchant_txn'] = (
    df_transactions.groupby('merchant_id')['timestamp']
    .diff()
    .dt.total_seconds()
)

df_transactions['last_user_merchant_ts'] = df_transactions.groupby(['user_id', 'merchant_id'])['timestamp'].shift(1)
df_transactions['time_since_last_user_merchant'] = (df_transactions['timestamp'] - df_transactions['last_user_merchant_ts']).dt.total_seconds()

df_transactions.drop(columns=['last_user_merchant_ts'], inplace=True)

#TODO: time since last transaction: user-merchant level

### Spatial feature engineering

In [88]:
from math import radians, sin, cos, sqrt, atan2
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # distance in km

In [89]:
# --------- Geo-data feature engineering
# Extract latitude and longitude into separate columns
df_transactions['latitude'] = df_transactions['location'].apply(lambda x: x['lat'])
df_transactions['longitude'] = df_transactions['location'].apply(lambda x: x['long'])
df_transactions.drop(columns=['location'], inplace=True)

df_transactions.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Shift lat/lon to get last user's lat/lon
df_transactions['prev_latitude'] = df_transactions.groupby('user_id')['latitude'].shift(1)
df_transactions['prev_longitude'] = df_transactions.groupby('user_id')['longitude'].shift(1)

df_transactions['distance_from_last_txn_km'] = df_transactions.apply(
    lambda row: haversine_distance(row['prev_latitude'], row['prev_longitude'],
                                   row['latitude'], row['longitude']),
    axis=1
)

df_transactions['velocity_kmph'] = df_transactions.apply(
    lambda row: (
        row['distance_from_last_txn_km'] / (row['time_since_last_user_txn'] / 3600)
        if row['time_since_last_user_txn'] > 0 else np.nan
    ),
    axis=1
)

df_transactions.head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,payment_method,is_international,...,month,time_since_last_user_txn,time_since_last_merchant_txn,time_since_last_user_merchant,latitude,longitude,prev_latitude,prev_longitude,distance_from_last_txn_km,velocity_kmph
290822,TX290822,2022-01-08 10:03:00,U00001,M0906,19.09,online,EUR,Web,debit_card,1,...,1,NaN,204900.0,NaN,69.291741,-7.726250,NaN,NaN,NaN,NaN
197951,TX197951,2022-01-08 16:48:00,U00001,M0245,32.24,online,EUR,Android,mobile_payment,1,...,1,24300.0,1140.0,NaN,54.710715,15.368695,69.291741,-7.726250,1993.988051,295.405637
187979,TX187979,2022-01-12 16:23:00,U00001,M0332,4.29,in-store,EUR,Android,debit_card,1,...,1,344100.0,71940.0,NaN,58.897717,30.255430,54.710715,15.368695,1015.860215,10.628006
137034,TX137034,2022-01-18 10:40:00,U00001,M0331,67.00,in-store,EUR,iOS,debit_card,1,...,1,497820.0,81780.0,NaN,42.684202,3.843828,58.897717,30.255430,2556.056611,18.484199
401318,TX401318,2022-07-18 08:05:00,U00001,M0971,3.59,in-store,EUR,Web,mobile_payment,1,...,7,15629100.0,76920.0,NaN,44.296089,15.749937,42.684202,3.843828,976.137696,0.224843


In [125]:
df_users.rename(columns = lambda x: x + '_user' if x != 'user_id' else x, inplace=True)
df_merchants.rename(columns = lambda x: x + '_merchant' if x != 'merchant_id' else x, inplace=True)

df_merged = df_transactions.merge(df_users, on='user_id', how='left')
df_merged = df_merged.merge(df_merchants, on='merchant_id', how='left')

In [135]:
df_merged.head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,payment_method,is_international,...,country_user,signup_date_user,risk_score_user,category_merchant,country_merchant,trust_score_merchant,number_of_alerts_last_6_months_merchant,avg_transaction_amount_merchant,account_age_months_merchant,has_fraud_history_merchant
0,TX290822,2022-01-08 10:03:00,U00001,M0906,19.09,online,EUR,Web,debit_card,1,...,Finland,2021-04-01,0.571079,grocery,Belgium,0.631421,2,26.85,37,0
1,TX197951,2022-01-08 16:48:00,U00001,M0245,32.24,online,EUR,Android,mobile_payment,1,...,Finland,2021-04-01,0.571079,gaming,Spain,0.859810,2,197.43,115,1
2,TX187979,2022-01-12 16:23:00,U00001,M0332,4.29,in-store,EUR,Android,debit_card,1,...,Finland,2021-04-01,0.571079,grocery,Spain,0.934815,5,18.36,68,0
3,TX137034,2022-01-18 10:40:00,U00001,M0331,67.00,in-store,EUR,iOS,debit_card,1,...,Finland,2021-04-01,0.571079,gaming,Belgium,0.859404,6,123.91,94,1
4,TX401318,2022-07-18 08:05:00,U00001,M0971,3.59,in-store,EUR,Web,mobile_payment,1,...,Finland,2021-04-01,0.571079,clothing,Germany,0.516453,1,0.98,52,1
